In [5]:
import cv2
import glob
import matplotlib
import numpy as np
import os
import torch

from depth_anything_v2.dpt import DepthAnythingV2

DEVICE = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'

model_configs = {
    'vits': {'encoder': 'vits', 'features': 64, 'out_channels': [48, 96, 192, 384]},
    'vitb': {'encoder': 'vitb', 'features': 128, 'out_channels': [96, 192, 384, 768]},
    'vitl': {'encoder': 'vitl', 'features': 256, 'out_channels': [256, 512, 1024, 1024]},
    'vitg': {'encoder': 'vitg', 'features': 384, 'out_channels': [1536, 1536, 1536, 1536]}
}

depth_anything = DepthAnythingV2(**model_configs['vits'])
depth_anything.load_state_dict(torch.load(f'checkpoints/depth_anything_v2_vits.pth', map_location='cpu'))
depth_anything = depth_anything.to(DEVICE).eval()

C:\Users\admin\AppData\Local\Temp\ipykernel_1100\4145113155.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  depth_anything.load_state_dict(torch.load(f'checkpoints/dept

In [6]:
sample_input = torch.rand(1, 3, 224, 224, device=DEVICE)
traced_model = torch.jit.trace(depth_anything, sample_input)

f:\work\2025\3_upwork_coreml\Vision\Depth-Anything-V2\depth_anything_v2\dinov2_layers\patch_embed.py:73: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert H % patch_H == 0, f"Input image height {H} is not a multiple of patch height {patch_H}"
f:\work\2025\3_upwork_coreml\Vision\Depth-Anything-V2\depth_anything_v2\dinov2_layers\patch_embed.py:74: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert W % patch_W == 0, f"Input image width {W} is not a multiple of patch width: {patch_W}"
f:\work\2025\3_upwork_coreml\Vision\Depth-Anything-V2\depth_anythin

In [10]:
import coremltools as ct

mlmodel = ct.convert(
    traced_model,
    inputs=[ct.ImageType(name="input", shape=sample_input.shape)],
    minimum_deployment_target=ct.target.iOS15,
    convert_to="mlprogram",
    debug=True,  # Helps in getting more detailed error logs
)



Converting PyTorch Frontend ==> MIL Ops:   0%|          | 0/1070 [00:00<?, ? ops/s]

ERROR - converting 'upsample_bicubic2d' op (located at: 'patch_pos_embed.3'):

Converting PyTorch Frontend ==> MIL Ops:   8%|▊         | 88/1070 [00:00<00:00, 6065.12 ops/s]

the following model ops are IMPLEMENTED:
  _convolution
  add
  cat
  constant
  dropout
  expand
  flatten
  floor_divide
  gelu
  int
  layer_norm
  linear
  listconstruct
  matmul
  mul
  numtotensor
  permute
  relu
  relu_
  reshape
  select
  size
  slice
  softmax
  squeeze
  to
  transpose
  unsqueeze
  upsample_bilinear2d
  view
the following model ops are MISSING:
  upsample_bicubic2d


RuntimeError: PyTorch convert function for op 'upsample_bicubic2d' not implemented.